# Trigger

In [1]:
# Function to install and import packages
def install_and_import(package):
    import importlib
    try:
        importlib.import_module(package)
    except ImportError:
        import pip
        pip.main(['install', package])
    finally:
        globals()[package] = importlib.import_module(package)

In [2]:
# Packages needed
install_and_import("pyodbc")
install_and_import("pandas")

In [35]:
# Database server connection
conn = pyodbc.connect(
       DRIVER='{ODBC Driver 17 for SQL Server}',  # Check driver version in ODBC Data Source Administrator in your personal computer
       SERVER = 'DESKTOP-CC1UAQG\SQL2019',          # SQL server name that can be seen when lauch SQL server 
       DATABASE = 'Survey_Sample_A19', 
       trusted_Connection = 'Yes')

# Create cursor to traverse over the records in a database
cur = conn.cursor()

In [36]:
# Create class for trigger 

class Trigger:
      
    def __init__(self):
        print("Instance of Trigger called")
    
        
    # Function to get survey structure
    def getSurveyStructure(self, nameCSVinString, dbconn):
        query_SurveyStructure = pandas.read_sql_query("SELECT * FROM SurveyStructure ORDER BY SurveyId", dbconn)
        df_SurveyStructure = pandas.DataFrame(query_SurveyStructure)
        # Save original survey structure table in local drive
        df_SurveyStructure.to_csv(nameCSVinString, index=False)
        return df_SurveyStructure
    
    
    # Get all surveyId 
    def allSurveyId (self):
        query_ColSurveyId = "SELECT DISTINCT SurveyId FROM Answer ORDER BY SurveyId"
        cur.execute(query_ColSurveyId)
        row = [item[0] for item in cur.fetchall()]
        return row

    
    # Get all question Id
    def allQuestionId (self):
        query_ColQuestionId = "SELECT DISTINCT QuestionId FROM Answer ORDER BY QuestionId"
        cur.execute(query_ColQuestionId)
        row = [item[0] for item in cur.fetchall()]
        return row
    
    
    # Function to verify the question in each survey
    def AllSurvey_Question(self,SurveyList):
        checkQuestion_List = [ ]

        for surveyId in SurveyList:
            if surveyId < len(SurveyList):
                query_CheckQ_inSurvey = "SELECT * FROM (SELECT SurveyId, QuestionId, 1 as InSurvey FROM SurveyStructure WHERE " \
                                       "SurveyId = " +str(surveyId)+ \
                                       " UNION "  \
                                       "SELECT " +str(surveyId)+ " as SurveyId, Q.QuestionId, 0 as InSurvey" \
                                       " FROM Question as Q WHERE NOT EXISTS" \
                                       "(SELECT * FROM SurveyStructure as S WHERE S.SurveyId = " +str(surveyId)+\
                                       " AND S.QuestionId = Q.QuestionId)) as t UNION "
                checkQuestion = query_CheckQ_inSurvey 
                checkQuestion_List.append(checkQuestion)

            else: 
                query_CheckQ_LastSurvey = "SELECT * FROM (SELECT SurveyId, QuestionId, 1 as InSurvey FROM SurveyStructure WHERE " \
                                       "SurveyId = " +str(surveyId)+ \
                                       " UNION "  \
                                       "SELECT " +str(surveyId)+ " as SurveyId, Q.QuestionId, 0 as InSurvey" \
                                       " FROM Question as Q WHERE NOT EXISTS" \
                                       "(SELECT * FROM SurveyStructure as S WHERE S.SurveyId = " +str(surveyId)+\
                                       " AND S.QuestionId = Q.QuestionId)) as t ORDER BY SurveyId " 
                checkQuestion = query_CheckQ_LastSurvey
                checkQuestion_List.append(checkQuestion)


            allsurvey_question = " ".join(checkQuestion_List)
            surveyId  += 1
        return allsurvey_question
    
    
    # Create data frame to see all question in each query
    def getAllQuestioninSurvey(self,query, dbconn):
        query_allQuestioninSurvey = pandas.read_sql_query(query, dbconn)
        df_allQuestioninSurvey = pandas.DataFrame(query_allQuestioninSurvey)
        return df_allQuestioninSurvey

    
    # Make list of question in each survey 
    def getAllInsurveyQuestion(self,df):
        df_InSurvey = df.loc[df["InSurvey"]==1]
        InSurvey_list = df_InSurvey.values[:,[0,1]].tolist()
        return InSurvey_list


  # Select only user who gives response
    def Final(self,SurveyList, QuestionList, List):
        strCurrentUnionQueryBlock_List = [ ]
        strFinalQuery = " "

        # Match answer to question
        def DynamicAnswer(surveyId,questionId,List):
            strColumnsQueryPart = " "
            AnswerCol = "COALESCE((SELECT a.Answer_Value FROM Answer as a "\
                             "WHERE a.UserId = u.UserId AND a.SurveyId = " +str(surveyId)+\
                             " AND a.QuestionId = "+str(questionId)+"),-1) "\
                             "AS ANS_Q" +str(questionId)
            NullCol = " NULL AS ANS_Q" +str(questionId) 
            value = [surveyId,questionId]
            if value in List:
                strColumnsQueryPart = AnswerCol
            else:
                strColumnsQueryPart = NullCol
            return strColumnsQueryPart
           

        for surveyId in SurveyList:
            strQueryTemplateOuterUnionQuery = " SELECT UserId, " +str(surveyId)+ " as SurveyId, "
            for questionId in QuestionList:
                if questionId in range(0,len(QuestionList)):
                    strQueryTemplateOuterUnionQuery = strQueryTemplateOuterUnionQuery + DynamicAnswer(surveyId,questionId,List) + ","
                else: 
                    strQueryTemplateOuterUnionQuery = strQueryTemplateOuterUnionQuery + DynamicAnswer(surveyId,questionId,List) + " FROM [User] as u" \
                                                      " WHERE EXISTS (SELECT * FROM Answer as a WHERE u.UserId = a.UserId" \
                                                      " AND a.SurveyId = " +str(surveyId)+ ") UNION "

                    strCurrentUnionQueryBlock_List.append(strQueryTemplateOuterUnionQuery)
                    strCurrentUnionQueryBlock = " ".join(strCurrentUnionQueryBlock_List)
                questionId += 1
            surveyId += 1
            strFinalQuery = " ".join(strCurrentUnionQueryBlock.split(" ")[:-2])
        return strFinalQuery


    # Get final survey data
    def Output(self, query,dbconn, nameCSVinString):
        query_Output = pandas.read_sql_query(query, dbconn)
        df_Output = pandas.DataFrame(query_Output)
        df_Output.to_csv(nameCSVinString, index=False)
        return df_Output.head(5)   # Check output
    
    
    # Create view
    def createView(self, View_NameInString, query, dbcur, dbconn):
        query_view = "CREATE VIEW " + View_NameInString + " AS " + query
        dbcur.execute(query_view)
        dbconn.commit()
        
        
    # Drop view
    def dropView(self, View_NameInString, dbcur, dbconn):
        query_view = "DROP VIEW " + View_NameInString 
        dbcur.execute(query_view)
        dbconn.commit()
        
        
    # Close connection
    def dbClose(self, dbconn, dbcur):
        dbcur.close()
        del dbcur
        dbconn.close()
        
        
    def main(self):
        df_SurveyStructure = self.getSurveyStructure("SurveyStructure_Khwansiri_update.csv", conn)
        allSurveyId_list = self.allSurveyId()
        allQuestionId_list = self.allQuestionId()
        QuestionInSurvey = self.AllSurvey_Question(allSurveyId_list)
        df_AllQuestion =  self.getAllQuestioninSurvey(QuestionInSurvey,conn)
        InSurvey_list =  self.getAllInsurveyQuestion(df_AllQuestion)
        FinalQuery = self.Final(allQuestionId_list, allSurveyId_list, InSurvey_list)
        AllSurveyData = self.Output(FinalQuery, conn, "GetAllSurveyDataSQL_Khwansiri.csv")
        self.dropView("vw_AllSurveyData_Khwansiri", cur, conn)
        self.createView("vw_AllSurveyData_Khwansiri", FinalQuery, cur, conn)
        dbconn.close(conn,cur)

In [44]:
# Look at current survey structure table
query_Current_SurveyStructure = pandas.read_sql_query("SELECT * FROM SurveyStructure ORDER BY SurveyId", conn)
df_Current_SurveyStructure = pandas.DataFrame(query_Current_SurveyStructure)

# Load original survey structure table from local computer
df_Original_SurveyStructure = pandas.read_csv("E:\DataScience\DSTI\SQL\Exam\SurveyStructure_Khwansiri.csv")

In [45]:
db_A19_update = Trigger()

Instance of Trigger called


In [46]:
# Compare current survey structure and the original one 
def checkUpdate(originalTable, newTable,View_NameInString):
    if newTable.equals(originalTable) == False:
        print ("There are some updates in survey's structure >> View table is updated")
        db_A19_update.main()
        
    else:
        print("Survey's structure has no update >> No need to create view table")
        db_A19_update.dbClose(conn,cur)

In [47]:
checkUpdate(df_Original_SurveyStructure, df_Current_SurveyStructure,"vw_AllSurveyData_Khwansiri")

Survey's structure has no update >> No need to create view table
